# 1.2 Reading a 2D vector field from a simulation

First we define the objects we will use - the mesh reader (to read the mesh from the FEM file), the pickle manager (to save the mesh data that we want to analyse), and the unfolder (to manage the mesh).

In [4]:
import numpy as np
import sys
sys.path.insert(0,'/home/cbyers/projects/working_branches/cyclops/src')
from cyclops.sim_reader import MeshReader, Unfolder
from cyclops.object_reader import PickleManager
from cyclops.regressors import LModel, CSModel
from cyclops.fields import VectorField

# Define the necessary objects
reader = MeshReader("data/monoblock_out.e")
print(type(reader))
pickle_manager = PickleManager()
unfolder = Unfolder()

Loaded mesh:  <meshio mesh object>
  Number of points: 87200
  Number of cells:
    hexahedron27: 2016
    hexahedron27: 3360
    hexahedron27: 4704
  Point sets: right, top, left, , centre_x_bottom_y_back_z, centre_x_bottom_y_front_z, left_x_bottom_y_centre_z, right_x_bottom_y_centre_z
  Point data: disp_x, disp_y, disp_z, temperature
  Cell data: vonmises_stress
<class 'cyclops.sim_reader.MeshReader'>


Next we read the simulation data. Change the sensor_region from `right` to whatever you called the region where sensors can be placed in your FEM file.

We get an array of 3D positions of all the nodes in that region in the mesh. In the monoblock file they all have x=0, but in your file they could all have z=5 or y=3 or whatever. The `compress_2D` function removes the x components so that the array becomes an array of 2D positions. You will have to redefine this function if you are analysing a different face or surface.

The way the mesh reader works is if you have a vector `disp` then it splits it into 3 scalars `disp_x`, `disp_y`, and `disp_z`. You can find out the names of your vector by looking at the mesh info that is output by the code above (if you use a vector called something other than `disp`). We read the components of the vector and then join them together into a `VectorField`.

We then save this field.

In [2]:
# Read the simulation data
sensor_region = "right"
pos_3D = reader.read_pos(sensor_region)
pos_2D = unfolder.compress_2D(pos_3D)
bounds = unfolder.find_bounds(pos_2D)
grid = unfolder.generate_grid(bounds, 30, 30)

disp = np.array(
    [
        reader.read_scalar(sensor_region, "disp_x"),
        reader.read_scalar(sensor_region, "disp_y"),
        reader.read_scalar(sensor_region, "disp_z"),
    ]
).T
print(pos_2D)
print(disp)
disp_field = VectorField(LModel, bounds)
disp_field.fit_model(pos_2D, disp)

# Save the simulation data
pickle_manager.save_file("results/disp_plane_field.pickle", disp_field)
pickle_manager.save_file("results/disp_plane_points.pickle", grid)

now reading positions in...
[[0.     0.0135]
 [0.001  0.0135]
 [0.0005 0.0135]
 ...
 [0.0115 0.0215]
 [0.012  0.021 ]
 [0.0115 0.021 ]]
[[ 5.42377424e-05  3.28911344e-05 -2.36492004e-05]
 [ 5.43988560e-05  3.49743114e-05 -1.95260568e-05]
 [ 5.43301892e-05  3.39904317e-05 -2.15822030e-05]
 ...
 [ 9.36166119e-05  7.56636920e-05  4.08665791e-05]
 [ 9.10786507e-05  7.06711488e-05  4.32006640e-05]
 [ 9.11093409e-05  7.20019583e-05  3.95953852e-05]]
known_pos [[0.     0.0135]
 [0.001  0.0135]
 [0.0005 0.0135]
 ...
 [0.0115 0.0215]
 [0.012  0.021 ]
 [0.0115 0.021 ]]
known_vectors [5.42377424e-05 5.43988560e-05 5.43301892e-05 ... 9.36166119e-05
 9.10786507e-05 9.11093409e-05]
1125 1125
X shape is  (1125, 2)  Y shape is  (1125, 3)
known_pos [[0.     0.0135]
 [0.001  0.0135]
 [0.0005 0.0135]
 ...
 [0.0115 0.0215]
 [0.012  0.021 ]
 [0.0115 0.021 ]]
known_vectors [3.28911344e-05 3.49743114e-05 3.39904317e-05 ... 7.56636920e-05
 7.06711488e-05 7.20019583e-05]
1125 1125
X shape is  (1125, 2)  Y shap

The displacement field is **NOT** uniform in the horizontal direction, but it doesn't change much and we can pretend that it is uniform if we want to get a 1D vector field. The code below considers the 1D strip along the edge of the field.

In [3]:
# Convert the 2D field to a 1D line
pos1 = (bounds[0][0], bounds[0][1])
pos2 = (bounds[0][0], bounds[1][1])
line_2D = unfolder.generate_line(pos1, pos2, 50)
line_disps = disp_field.predict_values(line_2D)
line_1D = unfolder.compress_1D(line_2D)

bounds_1D = np.array([line_1D[0], line_1D[-1]])
new_line_field = VectorField(CSModel, bounds_1D)
new_line_field.fit_model(line_1D, line_disps)

# Save the new 1D line field
pickle_manager.save_file("results/disp_line_field.pickle", new_line_field)
pickle_manager.save_file("results/disp_line_points.pickle", line_1D)

known_pos [[0.        ]
 [0.00071429]
 [0.00142857]
 [0.00214286]
 [0.00285714]
 [0.00357143]
 [0.00428571]
 [0.005     ]
 [0.00571429]
 [0.00642857]
 [0.00714286]
 [0.00785714]
 [0.00857143]
 [0.00928571]
 [0.01      ]
 [0.01071429]
 [0.01142857]
 [0.01214286]
 [0.01285714]
 [0.01357143]
 [0.01428571]
 [0.015     ]
 [0.01571429]
 [0.01642857]
 [0.01714286]
 [0.01785714]
 [0.01857143]
 [0.01928571]
 [0.02      ]
 [0.02071429]
 [0.02142857]
 [0.02214286]
 [0.02285714]
 [0.02357143]
 [0.02428571]
 [0.025     ]
 [0.02571429]
 [0.02642857]
 [0.02714286]
 [0.02785714]
 [0.02857143]
 [0.02928571]
 [0.03      ]
 [0.03071429]
 [0.03142857]
 [0.03214286]
 [0.03285714]
 [0.03357143]
 [0.03428571]
 [0.035     ]]
known_vectors [ 1.32301275e-05  0.00000000e+00 -4.02898445e-06  1.32873703e-05
  3.39280895e-07 -4.01723258e-06  1.34161230e-05  6.81950742e-07
 -3.98909115e-06  1.36172239e-05  1.02987064e-06 -3.94484803e-06
  1.38972921e-05  1.38578276e-06 -3.88362629e-06  1.42850011e-05
  1.76144076e-0

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 50 and the array at index 1 has size 450